In [1]:
## import necessary libraries
import numpy as np
import cvxpy as cp
from utils.tools import *
from utils.sim import *

In [2]:
T0 = 200
T1 = 5
J = 30
rho1, rho2 = 0.2, 0.2
gamma = 0.2
sig2 = 1.0
sig = np.sqrt(sig2)
trt_effect = 5
stationary = 1
n_rep = 1000
DGP = 3
fixed_alpha = None
fixed_lam = None
pollution = None
scale = False
intercept = True

In [3]:
T01 = T0 + T1
F1 = np.random.normal(scale=np.sqrt(2.0), size=T01)
F2 = np.random.normal(scale=np.sqrt(2.0), size=T01)

F1 += 5
F2 += 5

lambda1 = np.arange(1, J + 1) / J
lambda2 = lambda1.copy()

q2, r2 = divmod(J, 2)
q3, r3 = divmod(J, 3)
w1 = np.ones(J) / J
w_beta = np.random.beta(0.2, 0.2, J)
w2 = (w_beta - 0.5) * 2/J
w3 = np.random.normal(0, 2/J, J)
w4 = np.concatenate((np.random.normal(0, 2/J, q3), [0]*(2*q3+r3)))
np.random.shuffle(w4)
w5 = np.concatenate((np.random.uniform(-1, 2, 3), [0]*(J-3)))
np.random.shuffle(w5)
w = np.outer(w1, (DGP == 1)) + np.outer(w2, (DGP == 2)) + \
    np.outer(w3, (DGP == 3)) + np.outer(w4, (DGP == 4)) + \
    np.outer(w5, (DGP == 5))
y = np.kron(np.ones(T01), lambda1) + np.kron(F1, np.ones(J)) + \
    np.kron(F2, lambda2)

eps = np.empty((T01, J))
for j in range(J):
    eps[:, j] = generate_ARMA_series(T01, rho1, gamma)

Y0 = y.reshape(T01, J) + eps
u = generate_ARMA_series(T01, rho2, gamma)
# if DGP == 6:
#     Y1 = np.zeros((T01, 1))
#     Y1[:T0] = Y0[:T0] @ w
#     Y1[T0:] = Y0[T0:] @ w * 0.95
# else: 

Y1 = Y0 @ w + sig * u.reshape(-1, 1)
Y1 = Y1.flatten()
Y1[T0:] += trt_effect

In [4]:
## separate into pre-treatment and post-treatment
Y0_pre = Y0[:T0,:]
Y0_post = Y0[T0:,:]
Y1_pre = Y1[:T0]
Y1_post = Y1[T0:]

## add some contamination to the data
if pollution is not None:
    Y0_post = pollute(Y0_post, w, pollution)

w_sc = sc(Y1_pre, Y0_pre)    

idxs1 = np.arange(T0)
idxs2 = idxs1.copy()
alpha_inf, lam_inf = param_selector(
    Y1_pre[idxs1], Y0_pre[idxs1], method='inf', 
    fixed_alpha=fixed_alpha, fixed_lam=fixed_lam, 
    scale=scale, intercept=intercept)
alpha_l1, lam_l1 = param_selector(
    Y1_pre[idxs1], Y0_pre[idxs1], method='l1', 
    fixed_alpha=fixed_alpha, fixed_lam=fixed_lam, 
    scale=scale, intercept=intercept)
alpha_l2, lam_l2 = param_selector(
    Y1_pre[idxs1], Y0_pre[idxs1], method='l2',
    fixed_alpha=fixed_alpha, fixed_lam=fixed_lam,
    scale=scale, intercept=intercept)
alpha_l1_inf, lam_l1_inf = param_selector(
    Y1_pre[idxs1], Y0_pre[idxs1], method='l1-inf',
    fixed_alpha=fixed_alpha, fixed_lam=fixed_lam,
    scale=scale, intercept=intercept)
alpha_l1_l2, lam_l1_l2 = param_selector(
    Y1_pre[idxs1], Y0_pre[idxs1], method='l1-l2', 
    fixed_alpha=fixed_alpha, fixed_lam=fixed_lam, 
    scale=scale, intercept=intercept)

w_inf = our(
    Y1_pre[idxs2], Y0_pre[idxs2], alpha_inf, lam_inf, 'inf', 
    scale=scale, intercept=intercept)
w_l1 = our(
    Y1_pre[idxs2], Y0_pre[idxs2], alpha_l1, lam_l1, 'l1',  
    scale=scale, intercept=intercept)
w_l2 = our(
    Y1_pre[idxs2], Y0_pre[idxs2], alpha_l2, lam_l2, 'l2', 
    scale=scale, intercept=intercept)
w_l1_inf = our(
    Y1_pre[idxs2], Y0_pre[idxs2], alpha_l1_inf, lam_l1_inf, 'l1-inf', 
    scale=scale, intercept=intercept)
w_l1_l2 = our(
    Y1_pre[idxs2], Y0_pre[idxs2], alpha_l1_l2, lam_l1_l2, 'l1-l2',
    scale=scale, intercept=intercept)

In [5]:
# Calculate RMSE function
def calculate_rmse(estimator, true_weights):
    return np.sqrt(np.mean((estimator - true_weights) ** 2))

# Print estimators and their RMSE
print("True Coefficients (w):\n", w.flatten())
print("L-inf Weights (w_inf):\n", w_inf[1:])
print("RMSE for L-inf Weights:", calculate_rmse(w_inf[1:], w.flatten()))

# print("L1 Weights (w_l1):\n", w_l1[1:])
print("RMSE for L1 Weights:", calculate_rmse(w_l1[1:], w.flatten()))

# print("L2 Weights (w_l2):\n", w_l2[1:])
print("RMSE for L2 Weights:", calculate_rmse(w_l2[1:], w.flatten()))

# print("L1-L2 Weights (w_l1_l2):\n", w_l1_l2[1:])
print("RMSE for L1-L2 Weights:", calculate_rmse(w_l1_l2[1:], w.flatten()))

# print("L1-Inf Weights (w_l1_inf):\n", w_l1_inf[1:])
print("RMSE for L1-Inf Weights:", calculate_rmse(w_l1_inf[1:], w.flatten()))

True Coefficients (w):
 [-0.00886924  0.03463072  0.04818925 -0.02937985  0.06544347 -0.06192473
 -0.03407418  0.00905374  0.04107572  0.00708996  0.03510717  0.08861675
  0.02807487 -0.01681895 -0.05970153 -0.00135354  0.02109957  0.02274584
  0.12736701  0.0204307  -0.05380008  0.05834342  0.04088251 -0.08938538
  0.07048035  0.00933475 -0.19183677  0.01296348 -0.00920335 -0.02080039]
L-inf Weights (w_inf):
 [ 0.00887828  0.00887828  0.00887828  0.00887828  0.00887828  0.00887828
  0.00887828  0.00887828  0.00887828 -0.00887814  0.00887828  0.00887828
  0.00887828 -0.00887827 -0.00137957  0.00887828  0.00887828  0.00887828
  0.00887828  0.00887828 -0.00887828  0.00887828  0.00887828 -0.00887828
  0.00887828  0.00887828 -0.00887828 -0.00887828  0.00887828 -0.00887828]
RMSE for L-inf Weights: 0.0550819055121801
RMSE for L1 Weights: 0.058072945960526055
RMSE for L2 Weights: 0.046858402544701215
RMSE for L1-L2 Weights: 0.05717591709166848
RMSE for L1-Inf Weights: 0.057353175687738726


In [6]:
if intercept:
    Y0_post_plus = np.hstack([np.ones((Y1_post.shape[0], 1)), Y0_post])
else:
    Y0_post_plus = Y0_post
# tau_l1asso = Y1_post - Y0_post_plus @ w_l1asso

tau_oracle = Y1_post - Y0_post @ w.flatten()
tau_sc = Y1_post - Y0_post @ w_sc
tau_l1 = Y1_post - Y0_post_plus @ w_l1
tau_l2 = Y1_post - Y0_post_plus @ w_l2
tau_inf = Y1_post - Y0_post_plus @ w_inf
tau_l1_l2 = Y1_post - Y0_post_plus @ w_l1_l2
tau_l1_inf = Y1_post - Y0_post_plus @ w_l1_inf

print("Synthetic Control Estimator (tau_sc):", tau_sc)
print("L1 Estimator (tau_l1):", np.mean(tau_l1))
print("L2 Estimator (tau_l2):", np.mean(tau_l2))
print("Inf Estimator (tau_inf):", np.mean(tau_inf))
print("L1-L2 Estimator (tau_l1_l2):", np.mean(tau_l1_l2))
print("L1-Inf Estimator (tau_l1_inf):", np.mean(tau_l1_inf))

Synthetic Control Estimator (tau_sc): [ 2.17368651 -1.96510371  1.73041996 -0.26132564 -1.85667724]
L1 Estimator (tau_l1): 5.080779439928782
L2 Estimator (tau_l2): 5.140997790746549
Inf Estimator (tau_inf): 5.082551810715111
L1-L2 Estimator (tau_l1_l2): 5.0844217498786906
L1-Inf Estimator (tau_l1_inf): 5.105522645195549
